In [117]:
import regex
import csv
import numpy as np
import pandas as pd
import spacy
#from spacy.en import English
from nltk import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
#from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords

In [143]:
airline_df = pd.read_csv("../data/raw/twitter-airline-sentiment/Tweets.csv")

In [ ]:
#### Remove text field

In [144]:
airline_df.drop("text", axis = 1, inplace = True)

In [138]:
airline_df.columns

Index([u'tweet_id', u'airline_sentiment', u'airline_sentiment_confidence',
       u'negativereason', u'negativereason_confidence', u'airline',
       u'airline_sentiment_gold', u'name', u'negativereason_gold',
       u'retweet_count', u'tweet_coord', u'tweet_created', u'tweet_location',
       u'user_timezone'],
      dtype='object')

In [ ]:
#### Get unique values for categorical columns

In [7]:
[(col, airline_df[col].unique()) for col in ["airline_sentiment",\
                                                      "negativereason",\
                                                       "airline_sentiment_gold",\
                                                       "negativereason_gold"]]

[('airline_sentiment',
  array(['neutral', 'positive', 'negative'], dtype=object)),
 ('negativereason', array([nan, 'Bad Flight', "Can't Tell", 'Late Flight',
         'Customer Service Issue', 'Flight Booking Problems',
         'Lost Luggage', 'Flight Attendant Complaints', 'Cancelled Flight',
         'Damaged Luggage', 'longlines'], dtype=object)),
 ('airline_sentiment_gold',
  array([nan, 'negative', 'neutral', 'positive'], dtype=object)),
 ('negativereason_gold',
  array([nan, 'Late Flight\nFlight Attendant Complaints',
         'Late Flight\nLost Luggage', 'Late Flight',
         'Customer Service Issue', "Can't Tell",
         'Cancelled Flight\nCustomer Service Issue', 'Cancelled Flight',
         'Bad Flight', 'Lost Luggage\nDamaged Luggage',
         'Late Flight\nCancelled Flight', 'Flight Attendant Complaints',
         'Customer Service Issue\nLost Luggage',
         "Customer Service Issue\nCan't Tell"], dtype=object))]

In [ ]:
#### Sentiment distribution

In [16]:
sentiment_df["count"].apply(lambda x: x/sum(x))

AttributeError: 'long' object has no attribute 'sum'

In [15]:
sentiment_df["count"].astype("float")

0    9178.0
1    3099.0
2    2363.0
Name: count, dtype: float64

In [31]:
sentiment_df = airline_df.groupby(by = "airline_sentiment" \
                                 ).size().reset_index(name = "count")
sentiment_df["%"] = np.round(sentiment_df["count"].astype("float")/np.sum(sentiment_df["count"]) \
                          * 100, 2)
            

In [32]:
sentiment_df

,airline_sentiment,count,%
0,negative,9178,62.69
1,neutral,3099,21.17
2,positive,2363,16.14


Negative sentiment as expected dominates with  63% of tweets, but the data is based on negative sentiment

In [ ]:
#### Replace missing values for airline sentiment with Unknown

In [146]:
airline_df["negativereason"].fillna("Unknown", inplace = True)

In [ ]:
#### Distribution of reasons for negative sentiment

In [148]:
negreason_df = airline_df.groupby(by = "negativereason" \
                                 ).size().reset_index(name = "count").sort_values("count", ascending = False)
negreason_df["%"] = np.round(negreason_df["count"].astype("float")/np.sum(negreason_df["count"]) \
                          * 100, 2)
negreason_df

,negativereason,count,%
9,Unknown,5462,37.31
3,Customer Service Issue,2910,19.88
7,Late Flight,1665,11.37
1,Can't Tell,1190,8.13
2,Cancelled Flight,847,5.79
8,Lost Luggage,724,4.95
0,Bad Flight,580,3.96
6,Flight Booking Problems,529,3.61
5,Flight Attendant Complaints,481,3.29
10,longlines,178,1.22


In [ ]:
#### If Unknown negative removed

In [149]:
negreason_df2 = airline_df[airline_df["negativereason"] != "Unknown"].groupby(by = "negativereason") \
                                   .size().reset_index(name = "count").sort_values("count", ascending = False)
negreason_df2["%"] = np.round(negreason_df2["count"].astype("float")/np.sum(negreason_df2["count"]) \
                          * 100, 2)
negreason_df2

,negativereason,count,%
3,Customer Service Issue,2910,31.71
7,Late Flight,1665,18.14
1,Can't Tell,1190,12.97
2,Cancelled Flight,847,9.23
8,Lost Luggage,724,7.89
0,Bad Flight,580,6.32
6,Flight Booking Problems,529,5.76
5,Flight Attendant Complaints,481,5.24
9,longlines,178,1.94
4,Damaged Luggage,74,0.81


In [ ]:
#### Get an indication of the number of tweets from 
#### gold customers

In [150]:
gold_df = airline_df.groupby(by = "airline_sentiment_gold" \
                                 ).size().reset_index(name = "count")
gold_df

,airline_sentiment_gold,count
0,negative,32
1,neutral,3
2,positive,5


In [ ]:
Only 40 out of 14,600 tweets from gold customers

In [ ]:
#### Create a column to signify if customer is classified as gold

airline_df["gold"] = ["True" for sentiment in airline_df["airline_sentiment_gold"] if sentiment == "Unknown"]

In [ ]:
#### Number of unique customers in the dataset

In [151]:
print("No of unique customers in dataset: ")
airline_df["name"].nunique()

No of unique customers in dataset: 


7701

In [ ]:
#### Top ten customers by complaint

In [152]:
customers10_df = airline_df.groupby(by \
                                    = "tweet_id").size().reset_index(name = "count").sort_values("count", ascending = False).head(10)

customers10_df           

,tweet_id,count
1748,5.696040e+17,28
1811,5.696670e+17,28
2394,5.702720e+17,26
1771,5.696270e+17,26
1815,5.696710e+17,25
1764,5.696200e+17,25
2393,5.702710e+17,24
1816,5.696720e+17,24
2074,5.699390e+17,23
1843,5.696990e+17,23


Steady decrease in number of complaints per customer. 
Check mean and median values for comparison.

In [153]:
customers_df = airline_df.groupby(by \
                                    = "tweet_id").size().reset_index(name = "count").sort_values("count", ascending = False).head(10)



In [154]:
print("Mean number of complaints per customer:  %f" %(customers_df["count"].mean()))
print("Median number of complaints per customer: %f" %(customers_df["count"].median()))

Mean number of complaints per customer:  25.200000
Median number of complaints per customer: 25.000000


Even distribution in the number of complaints per customer i.e. outliers 
are not an issue.

In [ ]:
#### Descriptive statistics on numerical fields

In [157]:
airline_df.describe()[["airline_sentiment_confidence", "negativereason_confidence", \
                     "retweet_count"]]

,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,14640.000000,10522.000000,14640.000000
mean,0.900169,0.638298,0.082650
std,0.162830,0.330440,0.745778
min,0.335000,0.000000,0.000000
25%,0.692300,0.360600,0.000000
50%,1.000000,0.670600,0.000000
75%,1.000000,1.000000,0.000000
max,1.000000,1.000000,44.000000


Confidence levels are high in the overall sentiment data = 1 (median), but the confidence in the resons for negative sentiment on average is 0.67 (median)

In [ ]:
#### Airlines with the most complaints

In [158]:
airline_company_df = airline_df.groupby(by \
                                    = "airline").size().reset_index(name = "count").sort_values("count", ascending = False)#.head(10)

airline_company_df           

,airline,count
4,United,3822
3,US Airways,2913
0,American,2759
2,Southwest,2420
1,Delta,2222
5,Virgin America,504


In [ ]:
#### Airlines with the most complaints

In [188]:
user_timezone_df = airline_df.groupby(by \
                                    = "user_timezone").size().reset_index(name = "count").sort_values("count", ascending = False).head(10)

user_timezone_df["%"] = np.round(user_timezone_df["count"].astype("float")/np.sum(user_timezone_df["count"]) \
                          * 100, 2)
user_timezone_df           

,user_timezone,count,%
31,Eastern Time (US & Canada),3744,41.03
27,Central Time (US & Canada),1931,21.16
62,Pacific Time (US & Canada),1208,13.24
67,Quito,738,8.09
12,Atlantic Time (Canada),497,5.45
57,Mountain Time (US & Canada),369,4.04
10,Arizona,229,2.51
49,London,195,2.14
2,Alaska,108,1.18
76,Sydney,107,1.17


Around 75% of the complaints occur in the Eastern, Central and Pacific 
timezones

In [ ]:
#### Time series data - extract hour, day, month, year, day of week, bdate,

In [165]:
airline_df["tweet_created_dt"] = pd.to_datetime(airline_df["tweet_created"])

In [167]:
airline_df["date"] = airline_df["tweet_created_dt"].dt.date
airline_df["day"] = airline_df["tweet_created_dt"].dt.day
#airline_df["dayofweek"] = airline_df["tweet_created_dt"].dt.day_name
airline_df["month"] = airline_df["tweet_created_dt"].dt.month
airline_df["year"] = airline_df["tweet_created_dt"].dt.year
airline_df["time"] = airline_df["tweet_created_dt"].dt.time
airline_df["hour"] = airline_df["tweet_created_dt"].dt.hour

In [184]:
print("Range of dates: ")
min(airline_df["date"]), max(airline_df["date"])

Range of dates: 


(datetime.date(2015, 2, 16), datetime.date(2015, 2, 24))

Data collected over a 9 day period. Any variations in sentiment etc. will likely be in hour (time) complaint occured.

Insert any bivariate and multivariate analysis at the respective points